# Markov Reward Processes

Markov Reward Processes are essentially just Markov chain, but every time the agent leaves a state it gets a reward.

<img src="/files/markov_reward_process.png", width=500px>

The gamma-weighted sum of all rewards for a single trajectory is called the return:

<img src="/files/return.png", width=500px>

In [366]:
from numpy.random import choice
import numpy as np

state_names = ["C1", "C2", "C3", "Pass", "Pub", "FB", "Sleep"]

p_matrix = [[0, .5, 0, 0, 0, .5, 0],
            [0, 0, .8, 0, 0, 0, .2],
            [0, 0, 0, .6, .4, 0, 0],
            [0, 0, 0, 0, 0, 0, 1],
            [.2, .4, .4, 0, 0, 0, 0],
            [.1, 0, 0, 0, 0, .9, 0],
            [0, 0, 0, 0, 0, 0, 0]]

_rewards = [-2, -2, -2, 10, 1, -1, 0]

# Just cus
class ProbabilityMatrixException(Exception):
    def __init__(self, message):
        self.message = message
    
class RewardChain:
    def __init__(self, p_matrix, rewards, state_names, terminal_index=6):
        self.p_matrix = p_matrix
        self.state_names = state_names
        self.terminal_index = terminal_index
        self.rewards = rewards
        
        assert (len(set([len(row) for row in p_matrix])) == 1), "p_matrix rows must have equal lengths!"
        assert len(p_matrix[0]) == len(p_matrix), "p_matrix must be square!"
        assert (len(self.rewards) == len(p_matrix)), "rewards must be same length as p_matrix"
    
    # Generates a random path through the chain
    def sample(self, start_state):
        path = []
        if isinstance(start_state, str):
            start_indx = self.state_names.index(start_state)
        else:
            start_indx = start_state
            
        state = start_indx
        
        while state != self.terminal_index:
            path.append(state)
            transition_p = self.p_matrix[state]
            
            # numpy.random.choice accepts choices (states 0 through len(p_matrix)) and
            # equally sized list of probabilities for those choices
            next_state = choice(range(len(self.p_matrix)), p=transition_p)
            state = next_state
        
        path.append(self.terminal_index)
        
        if isinstance(start_state, str):
            return self.pretty(path)
        else:
            return path
    
    
    def pretty(self, path):
        return [self.state_names[i] for i in path]
    
    
    def G(self, path, gamma):
        str_check = [isinstance(x, str) for x in path]
        if any(str_check):
            assert all(str_check), "Path must be all int or all string"
            path = [self.state_names.index(x) for x in path]
        
        counter = 0
        reward = 0
        for state in path:
            r = self.rewards[state] * (gamma**counter)
            reward += (self.rewards[state] * (gamma**counter))
            counter += 1
        return reward
        
        
        
        
            

In [372]:
chain = RewardChain(p_matrix, _rewards, state_names)
path = chain.sample("C1")
print("Return: ", chain.G(path, 1))

Return:  -20


## Expected Return

Since the return from any single trajectory is a random value, we can ask what the *expected* return is for a given state. This expectation is called the **value function**

<img src="/files/mrp_value_function.png", width=500px>

A rough estimate of a state's value function can be calculated by taking many samples and averaging the return from each:

In [379]:
# Single state estimate

rewards = []
state = "C1"
gamma = 1
for i in range(0, 1000):
    path = chain.sample(state)
    reward = chain.G(path, gamma)
    rewards.append(reward)

print('Average Return for {0}: {1:.2f}'.format(state, np.mean(rewards)))
    

Average Return for C1: -12.18


In [387]:
# Estimate for all states

# iterations = 100000
iterations = 5000

states = np.zeros(len(p_matrix))
for state in range(len(states)):
    rewards = []
    for i in range(0, iterations):
        path = chain.sample(state)
        reward = chain.G(path, 1)
        rewards.append(reward)
    
    states[state] = np.mean(rewards)

{state_names[j]: states[j] for j in range(len(p_matrix))}


{'C1': -12.49614,
 'C2': 1.43329,
 'C3': 4.3600300000000001,
 'FB': -22.6419,
 'Pass': 10.0,
 'Pub': 0.82921999999999996,
 'Sleep': 0.0}

## Solving the Value Function Analytically

The value function (also known as the bellman equation) can be solved for analytically with a little bit of linear algebra.

<img src="/files/analytical_solution_1.png", width=200px>
<img src="/files/analytical_solution_2.png", width=300px>
<img src="/files/analytical_solution_3.png", width=200px>

In [249]:
gamma = 1
R = np.array(_rewards)
P = np.matrix(p_matrix)
I = np.identity(len(p_matrix))

solution = np.dot(np.linalg.inv((I-gamma*P)), R)
solution = solution.tolist()[0]
for state in range(len(state_names)):
    print(state_names[state], solution[state])
    

C1 -12.543209876543214
C2 1.4567901234567908
C3 4.320987654320986
Pass 10.0
Pub 0.8024691358024683
FB -22.543209876543223
Sleep 0.0


## Comparing Solution vs. Sampling Estimates

Comparing our sampled estimates with the closed-form solution, it looks like we get pretty close. Try increasing the `iteration` variable to some high number (and potentially wait a really long time) and see how close you can get it.

In [388]:
print("\tSampling\tSolution")
print("\t--------\t--------")
for j in range(len(states)):
    print("{state}:\t{0:0.3f}\t\t{1:0.3f}".format(states[j], solution[j], state=state_names[j]))

	Sampling	Solution
	--------	--------
C1:	-12.496		-12.543
C2:	1.433		1.457
C3:	4.360		4.321
Pass:	10.000		10.000
Pub:	0.829		0.802
FB:	-22.642		-22.543
Sleep:	0.000		0.000
